In [2]:
# import packages
import os
import glob
import numpy as np
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import auth

/Users/jasondavis/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Add user input
Fill in the variables in the cell below with the following information

- `experiment` = name of experiment (outermost collection) in Firebase
- `local_folder` = name of folder you want created locally and populated with data
- `credential_path` = path to .json Firebase SDK service account key (explained [here](https://firebase.google.com/docs/admin/setup))
- `firebase_project_name` = name of project in Firebase (click Project Overview in the Firebase console to see name)
- `groups` = array of group names (as set by userGroup in src/utils.js and found as collections in the 'subject' document in your Firebase), each enclosed in single quotes and separated by commas
- `rating_types` = array of rating types, each enclosed in single quotes and separated by commas (same as ratingTypes from src/utils.js)
- `movie_names` = array of movie names, as written in Firebase stimuli table (no spaces), each enclosed in single quotes and separated by commas
- `vid_lens` = dictionary of video durations, where each entry contains the movie name as key and the movie duration (in seconds) as value (e.g. {'movie1': 100, 'movie2': 150, 'movie3': 120}) 


In [30]:
# USER VARIABLES (FILL IN HERE)
experiment = 'wasabi-online'
local_folder = '/Users/jasondavis/Documents/MATLAB/state_affect_mapping_jad/wasabi_online/'
credential_path = '/Users/jasondavis/Documents/Downloads/continuous-rater-jad-firebase-adminsdk-2ie9g-0080d92b8a.json'
firebase_project_name = 'continuous-rater-jad'
groups = ['Kung Fury Study 3'] # possible to only have on group, or can use multiple for organizational purposes
rating_types = ['pleasant', 'unpleasant', 'calm', 'aroused', 'funny', 'happy', 'angry', 'sad', 'disgusted', 'afraid', 'surprised', 'neutral']
movie_names = ['ColdWar', 'KungFuryPart1', 'KungFuryPart2']
vid_lens = {'ColdWar':999,'KungFuryPart1': 931, 'KungFuryPart2': 931} # example values, fill in with your own

### Set up communication with Firebase

In [31]:
cred = credentials.Certificate(credential_path)
firebase_admin.initialize_app(cred, {
  'projectId': firebase_project_name,
})

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

### Initialize data structures and directories

In [32]:
# initialize data structures
subject_rating_dict = {}
movie_q_count_dict = {}
sub_to_mov = {}
mov_to_sub = {}

# initialize directory structure
directories = []
directories.append(f'{local_folder}/')
directories.append(f'{local_folder}/Long/')
directories.append(f'{local_folder}/Blanks/')
directories.append(f'{local_folder}/Subjects/')
directories.append(f'{local_folder}/Subjects/Incomplete/')
directories.append(f'{local_folder}/Subjects/Groups/')
directories.append(f'{local_folder}/Summary/')
directories.append(f'{local_folder}/Ratings/')
for movie in movie_names:
    for rating in rating_types:
        combo = movie + "-" + rating
        movie_q_count_dict[combo] = 0
        mov_to_sub[combo] = []
        directories.append(f'{local_folder}/Ratings/{combo}/')
        
for directory in directories:
    if not os.path.isdir(directory):
        os.mkdir(directory) 

### Create blank pandas dataframes of proper shape to fill in with rating data

In [33]:
# use provided movie durations to create blank dataframe of proper length for each movie
blanks_path = f'{local_folder}/Blanks/'
for movie in movie_names:
    curr_vid_len = vid_lens[movie] + 3 # add buffer time
    init_ratings = np.full(curr_vid_len, -1)
    curr_df = pd.DataFrame({'rating': init_ratings})
    curr_df.to_csv(os.path.join(blanks_path, f'{movie}.csv')) # write out to file

In [34]:
# read blank dataframes back in and store in dictionary
blanks_path = f'{local_folder}/Blanks/'
blank_pd_dict = {}
directory_list = glob.glob(os.path.join(blanks_path, '*.csv'))
for file in directory_list:
    curr_df = pd.read_csv(file)
    drop_df = curr_df.drop(labels='Unnamed: 0', axis=1)
    movie = os.path.basename(file).split('.')[0]
    blank_pd_dict[movie] = drop_df

### Determine which subjects completed task, and save subject info to .csv files

In [35]:
# create good and bad subject lists (based on completion)
good_id_master_list = []
bad_id_master_list = []

for group in groups:
    good_id_list = [] # stores participants who completed HIT
    bad_id_list = [] # stores participants who started but didn't complete HIT
    sub_list = []

    group_path = f'{experiment}/subjects/{group}'
    group_collection = db.collection(group_path)
    group_subs = group_collection.stream()

    for sub in group_subs:
        sub_dict = sub.to_dict()
        if 'currentState' in sub_dict: # check to see if subject even started HIT
            if sub_dict['currentState'] == 'debrief' or 'HIT_complete' in sub_dict: # only keep subs who finished
                good_id_list.append(sub.id)
                good_id_master_list.append(sub.id)
                curr = pd.Series(sub_dict)
                sub_list.append(curr)
                file_path = f'{local_folder}/Subjects/{sub.id}.csv'
                curr.to_csv(file_path)
        else:
            bad_id_list.append(sub.id)
            bad_id_master_list.append(sub.id)
            curr = pd.Series(sub_dict)
            file_path = f'{local_folder}/Subjects/Incomplete/{sub.id}.csv'
            curr.to_csv(file_path)

    group_df = pd.DataFrame(sub_list)
    group_df.to_csv(f'{local_folder}/Subjects/Groups/{group}.csv')

### Get all ratings from subjects who completed task and store locally

In [36]:
# loops over subjects and gets ratings from database to store in local dictionary
# also counts number of subjects that have rated each question
# links subjects to movies and movies to subjects
good_id_set = set(good_id_master_list) # removes repeats

for good_id in good_id_set: 
    sub_to_mov[good_id] = []
    movie_list = []
    collection_path = f'{experiment}/ratings/{good_id}'
    curr_sub_ratings = db.collection(collection_path)
    HITs = curr_sub_ratings.stream()

    curr_movie_dict = {}
    for HIT in HITs:
        sub_to_mov[good_id].append(HIT.id)
        mov_to_sub[HIT.id].append(good_id)
        curr_movie_dict[HIT.id] = HIT.to_dict()
        movie_q_count_dict[HIT.id] += 1

    if good_id in subject_rating_dict:
        updated = subject_rating_dict[good_id].append(curr_movie_dict)
        subject_rating_dict[good_id] = updated
    else:
        movie_list.append(curr_movie_dict)
        subject_rating_dict[good_id] = movie_list

movie_counts = pd.Series(movie_q_count_dict)
movie_counts.to_csv(f'{local_folder}/Summary/Movie_Counts.csv')

### Write out rating dictionary to .csv rating files

In [37]:
# writes file containing rating-timestamp pairs for each subject to folder for each movie-rating pairing
for good_id in good_id_set:
    curr_sub = subject_rating_dict[good_id]
    for dictionary in curr_sub:
        for movie_rating in dictionary:
            words = movie_rating.split('-')
            blank_mov_pd = blank_pd_dict[words[0]].copy()
            rating_dict = dictionary[movie_rating]
            count = 0
            print(len(rating_dict))
            for timestamp in rating_dict:
                    if timestamp.isdigit():
                        blank_mov_pd.iloc[int(timestamp)] = rating_dict[timestamp]
                    else:
                        continue
            file_path = f'{local_folder}/Ratings/{movie_rating}/{good_id}.csv'  
            blank_mov_pd.to_csv(file_path)      

97
267
347
2
2
1
3
3
2
3
2
1
69
64
39
115
95
36
2
2
1
2
2
1
2
2
1
3
46
17
17
16
7
34
16
8
2
9
3
2
2
1
331
134
81
45
17
4
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
2
9
2
2
1
2
2
1
2
2
1
2
6
3
2
2
1
3
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
3
2
13
32
20
23
17
37
32
56
37
52
4
9
25
67
78
4
5
3
3
2
3
29
159
201
19
4
1
21
7
11
68
2
18
2
2
1
2
2
1
2
2
1
2
8
8
2
2
1
239
88
58
2
2
1
228
221
284
8
9
5
2
5
5
2
2
1
59
65
6
5
5
1
54
35
15
14
7
8
8
166
47
3
16
6
2
2
1
2
120
43
2
2
1
2
14
16
2
13
1
2
2
8
63
56
90
2
2
1
2
9
13
219
157
142
2
2
1
2
2
1
2
2
1
8
7
8
2
2
1
36
21
27
2
2
1
186
157
106
6
7
7
44
23
26
2
2
1
2
2
1
2
2
1
2
2
9
193
143
117
245
67
48
2
2
1
19
12
5
16
27
52
2
2
1
2
14
3
43
57
56
2
2
1
9
6
6
2
2
1
5
8
13
2
2
1
2
2
2
157
120
128
45
108
97
21
111
77
44
34
27
102
147
97
2
2
1
2
4
8
26
4
25
2
2
1
2
2
1
307
241
272
7
3
3
2
2
1
56
4
25
2
193
174
2
2
1
2
2
1
2
2
1
131
28
21
2
2
1
2
2
1
2
2
1
2
2
1
2
2
22
2
2
7
2
2
9
8
7
1
191
208
111
332
226
251
3
3
1
2
2
1
2
2
1
4
3
2
80
72
67
2
2
1
14
2
7
2
15


### Combine subject and rating info together to create long format file

In [38]:
# make the base frame for appending
cols = ['workerId', 'movie', 'ratingType', 'HIT_complete', 'age', 'assignmentId', 'birth', 'consentStatus', 'currentState', \
'ethnicity', 'feedback', 'handed', 'hitId', 'nativeLang', 'race', 'sex', 'startTime', \
'userId', 'mostRecentTime', 'timeStamp', 'ratingScore']

base_frame = pd.DataFrame(np.nan, index=[0], columns=cols)

In [39]:
# this take individual local csvs and turns them into long format per movie per rating per subject
dir_list = glob.glob(f'{local_folder}/Ratings/*')
master_long = base_frame.copy()

for directory in dir_list:
    path = directory + '/*.csv'
    rating_list = glob.glob(path)
    for file in rating_list:
        sub_id = os.path.basename(file).split('.')[0]
        movie_rating = file.split('/')[2]
        movie = movie_rating.split('-')[0]
        rating = movie_rating.split('-')[0]       

        # this should check and not rewrite files that already exist (speeds up process)
        if not os.path.isfile(f'{local_folder}/Long/{movie}-{rating}-{sub_id}.csv'):
            subject_long = base_frame.copy()
            rating_pd = pd.read_csv(file)
            sub_path = f'{local_folder}/Subjects/{sub_id}.csv'
            if os.path.isfile(sub_path):
                sub_pd = pd.read_csv(sub_path)
                title_list = sub_pd['Unnamed: 0'].values
                rename_dict = {}
                counter = 0
                for title in title_list:
                    rename_dict[counter] = title
                    counter += 1   
                new_sub_pd = sub_pd.transpose().rename(columns=rename_dict).drop(['Unnamed: 0'])

                new_pd = base_frame.copy()
                new_pd['movie'] = movie
                new_pd['ratingType'] = rating

                for category in base_frame:
                    if category in new_sub_pd:
                        new_pd[category] = new_sub_pd[category].values

                timestamp_dict = rating_pd.transpose().drop(['Unnamed: 0'])
                copy_pd = new_pd.copy()
                prevScore = -1
                rating_counter = 0
                for timestamp in timestamp_dict:
                    ratingScore = timestamp_dict[timestamp].values
                    if ratingScore != -1:
                        prevScore = ratingScore
                    else:
                        ratingScore = prevScore

                    copy_pd['timeStamp'] = timestamp
                    copy_pd['ratingScore'] = ratingScore
                    subject_long = pd.concat([subject_long, copy_pd], ignore_index=True)

                subject_long = subject_long.drop([0])
                subject_long.to_csv(f'{local_folder}/Long/{movie}-{rating}-{sub_id}.csv')                   

In [40]:
# this appends all individual long format files into one giant long format panda
master_long = base_frame.copy()
long_list = glob.glob(f'{local_folder}/Long/*.csv')

for file in long_list:
    curr_pd = pd.read_csv(file)
    master_long = pd.concat([master_long, curr_pd])

master_long.drop([0])
master_long.to_csv(f'{local_folder}/master_long.csv')

In [41]:
# read in saved copy
master_long = pd.read_csv(f'{local_folder}/master_long.csv')
master_long.drop(labels=[0], inplace=True)
master_long.drop(labels=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
sub_list = master_long['workerId'].unique()
movie_list = master_long['movie'].unique()
rating_list = master_long['ratingType'].unique()
movie = movie_list[0]
rating = rating_list[0]

In [42]:
master_long

,workerId,movie,ratingType,HIT_complete,age,assignmentId,birth,consentStatus,currentState,ethnicity,...,handed,hitId,nativeLang,race,sex,startTime,userId,mostRecentTime,timeStamp,ratingScore
1,521709877.0,jasondavis,jasondavis,2025-06-11 12:22:50.326000+00:00,54.0,521709877.0,SOUTH AFRICA,signed,complete,not_hispanic,...,right,521709877.0,English,['Black / African-American'],female,2025-06-11 12:22:50.326000+00:00,KqVBrnp1VPfrgMQEjxps6d8X5wM2,NaN,0.0,50.0
2,521709877.0,jasondavis,jasondavis,2025-06-11 12:22:50.326000+00:00,54.0,521709877.0,SOUTH AFRICA,signed,complete,not_hispanic,...,right,521709877.0,English,['Black / African-American'],female,2025-06-11 12:22:50.326000+00:00,KqVBrnp1VPfrgMQEjxps6d8X5wM2,NaN,1.0,50.0
3,521709877.0,jasondavis,jasondavis,2025-06-11 12:22:50.326000+00:00,54.0,521709877.0,SOUTH AFRICA,signed,complete,not_hispanic,...,right,521709877.0,English,['Black / African-American'],female,2025-06-11 12:22:50.326000+00:00,KqVBrnp1VPfrgMQEjxps6d8X5wM2,NaN,2.0,50.0
4,521709877.0,jasondavis,jasondavis,2025-06-11 12:22:50.326000+00:00,54.0,521709877.0,SOUTH AFRICA,signed,complete,not_hispanic,...,right,521709877.0,English,['Black / African-American'],female,2025-06-11 12:22:50.326000+00:00,KqVBrnp1VPfrgMQEjxps6d8X5wM2,NaN,3.0,50.0
5,521709877.0,jasondavis,jasondavis,2025-06-11 12:22:50.326000+00:00,54.0,521709877.0,SOUTH AFRICA,signed,complete,not_hispanic,...,right,521709877.0,English,['Black / African-American'],female,2025-06-11 12:22:50.326000+00:00,KqVBrnp1VPfrgMQEjxps6d8X5wM2,NaN,4.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240886,405819037.0,jasondavis,jasondavis,2025-05-30 09:20:21.378000+00:00,31.0,405819037.0,South Africa,signed,complete,hispanic,...,right,405819037.0,English,['Black / African-American'],male,2025-05-30 09:20:21.378000+00:00,Cuys5RYuuNbdvEc5ITkUWhFSfEJ3,NaN,929.0,50.0
240887,405819037.0,jasondavis,jasondavis,2025-05-30 09:20:21.378000+00:00,31.0,405819037.0,South Africa,signed,complete,hispanic,...,right,405819037.0,English,['Black / African-American'],male,2025-05-30 09:20:21.378000+00:00,Cuys5RYuuNbdvEc5ITkUWhFSfEJ3,NaN,930.0,50.0
240888,405819037.0,jasondavis,jasondavis,2025-05-30 09:20:21.378000+00:00,31.0,405819037.0,South Africa,signed,complete,hispanic,...,right,405819037.0,English,['Black / African-American'],male,2025-05-30 09:20:21.378000+00:00,Cuys5RYuuNbdvEc5ITkUWhFSfEJ3,NaN,931.0,50.0
240889,405819037.0,jasondavis,jasondavis,2025-05-30 09:20:21.378000+00:00,31.0,405819037.0,South Africa,signed,complete,hispanic,...,right,405819037.0,English,['Black / African-American'],male,2025-05-30 09:20:21.378000+00:00,Cuys5RYuuNbdvEc5ITkUWhFSfEJ3,NaN,932.0,50.0
